In [1]:
require(dplyr)
require(tibble)
require(tidyr)
require(reshape2)
require(ggplot2)
require(colorRamps)

Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: tibble
Loading required package: tidyr
Loading required package: reshape2

Attaching package: ‘reshape2’

The following object is masked from ‘package:tidyr’:

    smiths

Loading required package: ggplot2
Loading required package: colorRamps


In [7]:
potZdata <- read.table("claXYZ_pbc_n250.dat",header=TRUE)
potZdata %>% head
potZdata$Z %>%summary
str(potZdata)
potZdata$newZ <- potZdata$Z

m_lower = 57   
m_upper = 66
# Z =[56,66] is good for negative 750mV

#potZdata$class <- (-2+((potZdata$newZ>m_lower)+(potZdata$newZ>(m_lower+m_upper)/2)+(potZdata$newZ>m_upper)))
potZdata$class <- (-1+(potZdata$newZ> m_lower)+(potZdata$newZ> m_upper))
potZdata %>% head
#This part is to do the classification for the POT Z-axis values,
#in which, below m_lower is -1, above m_upper is +1, 
#between m_lower and m_upper is 0.

potZdata$XY_ID <- (-4+((potZdata$X>60) + (potZdata$X <135) + (potZdata$Y > 102) +(potZdata$Y <170)))
potZdata %>% head


#potZdata = potZdata[potZdata$class ==0,] # 2500frames=600ns, 212ns, NBD through Cx26 channel, 885frames.
potZdata = potZdata[potZdata$Frame<2085,] # 2500frames=600ns, 212ns, NBD through Cx26 channel, 885frames.
str(potZdata)

Frame,MEASUREMENT,X,Y,Z
0,Vec_00000,177.0536,199.1626,25.40681
1,Vec_00000,148.7968,138.0634,105.53223
2,Vec_00000,149.6650,136.6408,109.99245
3,Vec_00000,147.5652,136.5758,117.05359
4,Vec_00000,151.1582,142.9804,121.84068
5,Vec_00000,150.6513,131.7733,127.71510


     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
  0.00011  22.45963  92.50873  73.42905 117.43644 143.59898 

'data.frame':	1273572 obs. of  5 variables:
 $ Frame      : int  0 1 2 3 4 5 6 7 8 9 ...
 $ MEASUREMENT: Factor w/ 612 levels "Vec_00000","Vec_00001",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ X          : num  177 149 150 148 151 ...
 $ Y          : num  199 138 137 137 143 ...
 $ Z          : num  25.4 105.5 110 117.1 121.8 ...


Frame,MEASUREMENT,X,Y,Z,newZ,class
0,Vec_00000,177.0536,199.1626,25.40681,25.40681,-1
1,Vec_00000,148.7968,138.0634,105.53223,105.53223,1
2,Vec_00000,149.6650,136.6408,109.99245,109.99245,1
3,Vec_00000,147.5652,136.5758,117.05359,117.05359,1
4,Vec_00000,151.1582,142.9804,121.84068,121.84068,1
5,Vec_00000,150.6513,131.7733,127.71510,127.71510,1


Frame,MEASUREMENT,X,Y,Z,newZ,class,XY_ID
0,Vec_00000,177.0536,199.1626,25.40681,25.40681,-1,-2
1,Vec_00000,148.7968,138.0634,105.53223,105.53223,1,-1
2,Vec_00000,149.6650,136.6408,109.99245,109.99245,1,-1
3,Vec_00000,147.5652,136.5758,117.05359,117.05359,1,-1
4,Vec_00000,151.1582,142.9804,121.84068,121.84068,1,-1
5,Vec_00000,150.6513,131.7733,127.71510,127.71510,1,-1


'data.frame':	1273572 obs. of  8 variables:
 $ Frame      : int  0 1 2 3 4 5 6 7 8 9 ...
 $ MEASUREMENT: Factor w/ 612 levels "Vec_00000","Vec_00001",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ X          : num  177 149 150 148 151 ...
 $ Y          : num  199 138 137 137 143 ...
 $ Z          : num  25.4 105.5 110 117.1 121.8 ...
 $ newZ       : num  25.4 105.5 110 117.1 121.8 ...
 $ class      : num  -1 1 1 1 1 1 1 1 1 1 ...
 $ XY_ID      : num  -2 -1 -1 -1 -1 -1 -1 -1 -1 -1 ...


In [8]:
potSelect <- filter(potZdata,potZdata$class== -1)
#potSelect %>% head
#potSelect$class[1]
#write.csv(potZdata, file="potZdata_class.csv")

potUniq <- subset(potZdata,select = -c(Z,newZ))
potUniq %>% head


potSort <- potUniq %>% 
    group_by(MEASUREMENT) %>%
     #top_n(n=-1,wt=Frame) 
## class[1]-1 and merge with class[1] to class[len(class)-1], shift 1 row to get change coloum
    mutate(class_nm=c(class[1]-1,class[c(1:(length(class)-1))])) %>%
    mutate(change=1.0*(abs(class-class_nm)>0)) %>%
    filter(change==1)
# This is to get rid of POT positions which did not change between classes, stayed in the same class.
   
potSort %>% head

Frame,MEASUREMENT,X,Y,class,XY_ID
0,Vec_00000,177.0536,199.1626,-1,-2
1,Vec_00000,148.7968,138.0634,1,-1
2,Vec_00000,149.6650,136.6408,1,-1
3,Vec_00000,147.5652,136.5758,1,-1
4,Vec_00000,151.1582,142.9804,1,-1
5,Vec_00000,150.6513,131.7733,1,-1


Frame,MEASUREMENT,X,Y,class,XY_ID,class_nm,change
0,Vec_00000,177.0536,199.1626,-1,-2,-2,1
1,Vec_00000,148.7968,138.0634,1,-1,-1,1
10,Vec_00000,144.7763,121.3107,-1,-1,1,1
12,Vec_00000,145.3211,127.9200,1,-1,-1,1
13,Vec_00000,146.2304,127.6255,-1,-1,1,1
26,Vec_00000,137.4186,129.3781,1,-1,-1,1


In [9]:
potSize <- potSort %>%
   group_by(MEASUREMENT) %>%
   summarise(n_size =length(MEASUREMENT))
potSize %>% head
str(potSize)

MEASUREMENT,n_size
Vec_00000,86
Vec_00001,124
Vec_00002,99
Vec_00003,117
Vec_00004,85
Vec_00005,110


Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	612 obs. of  2 variables:
 $ MEASUREMENT: Factor w/ 612 levels "Vec_00000","Vec_00001",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ n_size     : int  86 124 99 117 85 110 112 45 104 125 ...


In [12]:
i =1
timeLot = c()
id_Lot <- c()
tot_count =0
time_interval=0.024
tot_time =0
nLev =0
nCLA=612
#& potSort$XY_ID[i] == 0
for (j in 1:nCLA)  {
    nLev = nLev + potSize$n_size[j]
    #print(potSize$n_size[j])
    # & potSort$XY_ID[i]==0
   while (i< (nLev-2)) {
       #if (potSort$class[i] == -1 & potSort$class[i+1] ==0 & potSort$class[i+2]== 1 & potSort$XY_ID[i+1] == 0 ) {
       if (potSort$class[i] == 1 & potSort$class[i+1] ==0 & potSort$class[i+2]== -1 & potSort$XY_ID[i+1] == 0 ) {
        num =1
        tot_count = num + tot_count        
        id_Lot <- c(id_Lot,potSort$MEASUREMENT[i])         
        #print(potSort$MEASUREMENT[i+2])
        #print(paste("End Transition Event time",potSort$Frame[i+2]))
        #print(paste("Start time",potSort$Frame[i]))
        timeLot <- c(timeLot,(potSort$Frame[i+2]-potSort$Frame[i])*time_interval)
        tot_time=tot_time+(potSort$Frame[i+2]-potSort$Frame[i])*time_interval
        #print(j)
        }
    else {
        tot_count = tot_count
    }
    i=i+1
   }
    #print(nLev)
    #print(i)
   i=i+3
}
timeLot 
id_Lot 
#id_Lot
print(tot_count)
print(tot_time)
tot_time/tot_count

NULL

NULL

[1] 0
[1] 0


[1] NaN

In [11]:
newLot <- timeLot[timeLot >0]
newLot

length(newLot)
sum(newLot)
sum(newLot)/length(newLot)

NULL

[1] 0

[1] 0

[1] NaN